# Sinergise Summer School 2022

## EO Research Team ML part I


Welcome! This notebook will show you how to get your hands dirty with machine learning (ML). We will look at the basics of how to use the distribution properties to classify burned areas, and proceed to using existing ML tools to improve our classification.

## Decision Trees Intro

Decision trees are one of the simplest (and most explainable) machine learning models.  The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features.

Some advantages of decision trees are:
* simple to understand and to interpret (can be visualized)
* requires little data preparation (other techniques often require normalization, imputation)
* the cost of using DTs (i.e., predicting data) is logarithmic in the number of data points used to train the tree

The disadvantages of decision trees include:
* can be easy to overfit
* can be problematic on unbalanced data

![](https://regenerativetoday.com/wp-content/uploads/2022/04/dt.png)

## Homecooked decision tree

Let's try to use "brain learning" first. 

_Can we make our own decision tree using only one or two features?_

It's oftentimes surprising how good simple models can be and it also gives us a nice baseline to compare our machine learning model to.

In [ ]:
import pandas as pd

# use the same dataset as in Part I
df_path = "s3://eo-learn.sentinel-hub.com/workshops_data/summer_school_sinergise_2022/input_data/wildfires_single_scene_training_dataset.parquet"
df = pd.read_parquet(df_path)

# show content
df

Below is an example skeleton decision tree. 

Create your own by looking at the band distributions from Part I and specifying regions for the particular class:
- 0: Not burned
- 1: Burned


```python
def homecooked_decision_tree(row):
    if row.B08 <= 0.15:
        return 1  # Burned
    elif ...:
        if ...:
            ...
    elif ...:
        return 0 # Not burned
    return 0 # Not burned
```

In [ ]:
# TODO: MAKE YOUR OWN DECISION TREE
# No need to complicate, start with something very simple. You will be surprised at how good it can get.


def homecooked_decision_tree(row):
    if ...
        return ...
    elif ...:
        return ...
    return ...

### Evaluation

How did we do? _We don't know yet!_ 

We need to evaluate the model by looking at the results and coming up with some qualitative and quantitative ways of evaluating them.

#### Qualitative assessment

We've prepared a utility function which you can use to apply the homecooked algorithm and see the output on the EOPatches.

Compare the output of the algorithm to the mask! What do you think about the results?

In [ ]:
import numpy as np
from eolearn.core import EOPatch
from utils import plot_combo

# download from AWS S3 bucket
eop_path = "s3://eo-learn.sentinel-hub.com/workshops_data/summer_school_sinergise_2022/input_data/eopatches/eopatch_1"
eop = EOPatch.load(eop_path, lazy_loading=True)

# plot true color and reference mask
rgb = np.concatenate([eop.data["B04"], eop.data["B03"], eop.data["B02"]], axis=-1)
plot_combo(rgb, eop.mask_timeless["BURN_AREA"], brightness_factor=3.5, clip=(0, 1))

In [ ]:
import matplotlib.pyplot as plt
from utils import apply_function_to_eopatch, plot_mask

# apply homecooked algo on eopatch
eop = apply_function_to_eopatch(
    eopatch=eop,
    timestamp_index=1,
    function=homecooked_decision_tree,
    output_mask_name="HOMECOOKED_DECISION_TREE",
)

# compare reference mask with our homecooked algorithm
fig, axs = plt.subplots(ncols=2, figsize=(12, 5))
plot_mask(eop.mask_timeless["BURN_AREA"], ax=axs[0], title="Reference mask")
plot_mask(
    eop.mask_timeless["HOMECOOKED_DECISION_TREE"],
    ax=axs[1],
    title="Homecooked decision tree",
)

#### Quantitative assessment

But how to best check this in a __quantitative__ way? 

We could compare how well our algorithm output `HOMECOOKED_DECISION_TREE` agrees with the reference labels `BURN_AREA`!

This can be done with the _confusion matrix_, which shows the different prediction labels compared to the different reference labels.
This way one can extract the amounts of true-positives, true-negatives, false-positives, and false-negatives.

![cm](https://miro.medium.com/max/667/1*3yGLac6F4mTENnj5dBNvNQ.jpeg)

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

# extract reference and predictions
y_true = eop.mask_timeless["BURN_AREA"].ravel()
y_pred = eop.mask_timeless["HOMECOOKED_DECISION_TREE"].ravel()
cm_homecooked_eop = confusion_matrix(y_true=y_true, y_pred=y_pred)

fig, ax = plt.subplots(figsize=(5, 5))

# plot CM
plot_confusion_matrix(
    conf_mat=cm_homecooked_eop,
    show_normed=True,
    class_names=["Not burned", "Burned"],
    axis=ax,
)
ax.set_title("Homecooked Decision Trees CM - one EOPatch");

What do you think about the performance of the homecooked model?

How does it perform on the full training dataset? Do you understand the differences?

In [ ]:
# apply it on the FULL dataset! using df.apply()
df["HOMECOOKED_DECISION_TREE"] = df.apply(...)

y_true = df["BURN_AREA"]
y_pred = df["HOMECOOKED_DECISION_TREE"]
cm_homecooked = confusion_matrix(y_true=y_true, y_pred=y_pred)
cm_homecooked_norm = confusion_matrix(y_true=y_true, y_pred=y_pred, normalize="true")

fig, ax = plt.subplots(figsize=(5, 5))
plot_confusion_matrix(
    conf_mat=cm_homecooked,
    show_normed=True,
    class_names=["Not burned", "Burned"],
    axis=ax,
)
ax.set_title("Homecooked Decision Trees CM");

#### Other Evaluation Metrics

F1 score, precision, accuracy, recall, (check out `sklearn.metrics`, which has a ton of metrics).

What do they mean?

In [ ]:
# import other metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print(f"true-negative-rate: {cm_homecooked_norm[0,0]:.3f}")
print(f"false-positive-rate: {cm_homecooked_norm[0,1]:.3f}")
print(f"false-negative-rate: {cm_homecooked_norm[1,0]:.3f}")
print(f"true-positive-rate: {cm_homecooked_norm[1,1]:.3f}")

print(f"accuracy: {accuracy_score(y_true, y_pred):.3f}")
print(f"precision: {precision_score(y_true, y_pred):.3f}")
print(f"recall: {recall_score(y_true, y_pred):.3f}")
print(f"F1-score: {f1_score(y_true, y_pred):.3f}")

## Decision Trees

Now let's try using official tools for Decision Trees classification. 


### Train/Test Split

Before we proceed, **let's make a train/test split first.**

This is needed in order not to overfit the model to the dataset. The simplest train/test split is to just randomly split the dataset. 

Let's use a 80/20 train/test split, since this method is the simplest

_Can you think of a more appropriate method, considering we're dealing with geospatial data?_

In [ ]:
TRAIN_SIZE = 0.8  # size of the training set (fraction)
df["SPLIT"] = ...

df_train = df[df.SPLIT == "TRAIN"]
df_test = df[df.SPLIT == "TEST"]

### Training and prediction

In [ ]:
# select features to be considered
feature_columns = [...]
label_column = "BURN_AREA"

# extract values
features_train, labels_train = (
    df_train[feature_columns].values,
    df_train[label_column].values,
)
features_test, labels_test = (
    df_test[feature_columns].values,
    df_test[label_column].values,
)

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth=3, random_state=42)
clf = clf.fit(...)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

# plot tree using the utility function
tree.plot_tree(
    clf,
    filled=True,
    ax=ax,
    feature_names=feature_columns,
    class_names=["Not burned", "Burned"],
    impurity=False,
);

### Evaluation

Repeating processes from before

In [ ]:
from utils import apply_decision_trees_to_eopatch

# copy-pasta for convenience
clf = tree.DecisionTreeClassifier(max_depth=6, random_state=42)
clf = clf.fit(features_train, labels_train)

# apply decision trees on eopatch
eop = apply_decision_trees_to_eopatch(
    eopatch=eop,
    timestamp_index=1,
    classifier=clf,
    output_mask_name="DECISION_TREE",
    features=feature_columns,
)

# compare reference mask with our homecooked algorithm
fig, axs = plt.subplots(ncols=2, figsize=(12, 5))
plot_mask(eop.mask_timeless["BURN_AREA"], ax=axs[0], title="Reference mask")
plot_mask(
    eop.mask_timeless["DECISION_TREE"],
    ax=axs[1],
    title="Official decision tree",
)

In [ ]:
df["DECISION_TREE"] = ...

y_true = df["BURN_AREA"].values
y_pred = df["DECISION_TREE"].values
cm_dt = confusion_matrix(y_true=y_true, y_pred=y_pred)
cm_dt_norm = confusion_matrix(y_true=y_true, y_pred=y_pred, normalize="true")

fig, ax = plt.subplots(figsize=(5, 5))
plot_confusion_matrix(
    conf_mat=cm_dt, show_normed=True, class_names=["Not burned", "Burned"], axis=ax
)
ax.set_title("Decision Trees CM");

In [ ]:
# calculate F1-score for multiple depths (hyper-parameter optimization)
depth_list = list(range(1, 7))
scores = []
for depth in depth_list:
    # 1. define the classifier with dynamic depth
    # 2. fit the classifier
    # 3. predict on test features
    # 4. calculate score on test

    scores.append(...)

fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(depth_list, scores, "C0x-")
ax.set_xlabel("Max Depth of DT")
ax.set_ylabel("F1 score")
ax.set_title("F1 score for different max depth of DT")

## LGBM

LightGBM is a gradient boosting framework that uses tree based learning algorithms.

Pros:
* fast
* compatible with both small and large datasets
* parallelizable
* good handling of overfitting
* **tt can be exported into an evalscript and visualized in eo-browser.**

Cons:
* doesn't take spatial context into account (here neural networks would help)
* hard to interpret
* already kind of a black box

![](https://miro.medium.com/max/1000/1*qHbAsMNmdWQJkzm2SUA-8w.jpeg)

We can start with all available bands to see what we get. Later on we can adjust based on what we learned about the data / indices so far.

### Training and prediction

In [ ]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier()  # check definition for changing parameters!
clf.fit(features_train, labels_train, feature_name=feature_columns);

### Evaluation

In [ ]:
from utils import apply_lgbm_to_eopatch

# apply LGBM on eopatch
eop = apply_lgbm_to_eopatch(
    eopatch=eop,
    timestamp_index=1,
    classifier=clf,
    output_mask_name="LGBM",
)

# compare reference mask with our homecooked algorithm
fig, axs = plt.subplots(ncols=3, figsize=(12, 5))
plot_mask(eop.mask_timeless["BURN_AREA"], ax=axs[0], title="Reference mask")
plot_mask(
    eop.mask_timeless["LGBM"],
    ax=axs[1],
    title="LGBM Prediction",
)
plot_mask(eop.data_timeless["LGBM_PROBA"], ax=axs[2], title="LGBM Pseudo-Probability")

In [ ]:
df["LGBM"] = ...

y_true = df["BURN_AREA"].values
y_pred = df["LGBM"].values
cm_lgbm = confusion_matrix(y_true=y_true, y_pred=y_pred)

fig, ax = plt.subplots(figsize=(5, 5))
plot_confusion_matrix(
    conf_mat=cm_lgbm, show_normed=True, class_names=["Not burned", "Burned"], axis=ax
)
ax.set_title("LGBM CM");

A ROC curve is a receiver operating characteristic curve. The ROC curve is created by plotting the true positive rate (TPR) against the false positive rate (FPR) at various threshold settings.

It can be used to show the probabilities for the classifier to predict FP or FN cases at different thresholds.

In [ ]:
from sklearn.metrics import roc_curve

# copy-pasta for convenience
clf = LGBMClassifier()
clf.fit(features_train, labels_train, feature_name=feature_columns)

df["LGBM_PROBA"] = ...  # hint: predict_proba

y_true = df["BURN_AREA"].values
y_pred_proba = df["LGBM_PROBA"].values

fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(fpr, tpr, label="ROC Curve LGBM")

# plot the homecooked point
# fpr_homecooked = ...
# tpr_homecooked = ...
# ax.scatter(fpr_homecooked, tpr_homecooked, label="Homecooked Model", color="C02")

# plot the decision-tree point
# fpr_dt = ...
# tpr_dt = ...
# ax.scatter(fpr_dt, tpr_dt, label="Decision Trees Model", color="C03")

# ideal point
ax.scatter(0.0, 1.0, label="Ideal model", color="black", marker="X")
ax.set_xlabel("False Positive Rate (FPR)")
ax.set_ylabel("True Positive Rate(TPR)")
ax.legend()

ax.set_xlim([-0.005, 0.4])
ax.set_ylim([0.6, 1.01]);

Plot feature importance. Does it reflect what was seen in the data?

In [ ]:
import lightgbm

lightgbm.plot_importance(clf, figsize=(12, 5))

### Export model to EO Browser

LGBM is just a glorified decision tree. With some helper functions and black magic it's possible to convert it into an Evalscript and visualize it in the EO Browser. 

In [ ]:
import joblib

model_file = "wildfires_example_model.pkl"
evalscript_file = "wildfires_example_model_evalscript.js"

joblib.dump(clf, model_file)

# convert and export to evalscript - bash command
! python convert-model-to-custom-script.py --model $model_file --output $evalscript_file

Paste it to EO Browser!

https://sentinelshare.page.link/g4vL

What can you see?
* Does the visualization reflect the quality as indicated by the scores?
* Does it generalize well across different areas?
* Does it generalize well across different years?


Now it's time to play around and try to improve the model. Some ideas to get you started:
* We made a random train/test split. Is this the best we can do?
* Would adding indices (NDVI, NBR) to the features improve it?
* We used default LGBM parameters, hyper parameter optimization could be one of the things to try.
* Feel free to explore your own ideas.